## H-dence U-Net
---

参考
- 論文 [H-DenseUNet: Hybrid Densely Connected UNet for Liver and Tumor Segmentation from CT Volumes](https://arxiv.org/pdf/1709.07330.pdf)
- 公式 [GitHubリポジトリ](https://github.com/xmengli999/H-DenseUNet)

In [ ]:
!ls ../Preprocess/datasets/train-label/
!ls ../Preprocess/datasets/train-image/
!ls ../Preprocess/datasets/test-label/
!ls ../Preprocess/datasets/test-image/

### 1. Setup Datasets

In [ ]:
def read_mhd_files(path, excepts=None, choose=None):
    '''
    指定ディレクトリの mhd 形式のファイルを読み込み、
    1-0. 指定されたファイル (ex. 2番目) を除くファイルリスト作成
    1-1. 例の結合された配列 images の作成
    1-2. images の各スライスごとに症例番号・スライス番号を格納した image_filesの作成
    '''

    if (excepts!=None):
        
        mhd_files = natsorted(glob.glob(path+'*.mhd'))
        del mhd_files[excepts]
        del mhd_files[excepts+1]
        del mhd_files[excepts+2]

        # 1症例目のみ配列作成のため、単体で読み込み
        images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
        image_files = []
        for i in range(images.shape[0]):
            image_files.append(mhd_files[0].split('/')[-1].split('.')[0] +'-'+str(i))    

        # 以降の症例
        for mhd_name in mhd_files:
            mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
            images = np.concatenate([images,mhd_array])
            for i in range(mhd_array.shape[0]):
                image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))     
    
    '''
    指定ディレクトリの mhd 形式のファイルを読み込み、
    1-0. 指定されたファイル ex. 2番目 のみの配列 images の作成
    1-1. images の各スライスごとに症例番号・スライス番号を格納した image_filesの作成
    '''
    if(choose!=None):

        mhd_files = natsorted(glob.glob(path+'*.mhd'))
        mhd_files = mhd_files[choose:(choose+2)]
        
        # 1症例目のみ配列作成のため、単体で読み込み
        images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
        image_files = []
        for i in range(images.shape[0]):
            image_files.append(mhd_files[0].split('/')[-1].split('.')[0] +'-'+str(i))    
            
        # 以降の症例
        for mhd_name in mhd_files:
            mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
            images = np.concatenate([images,mhd_array])
            for i in range(mhd_array.shape[0]):
                image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))     


    return images

def save_mhd_files(np_array, file_path):
    img = sitk.GetImageFromArray(np_array,isVector=False)
    img.SetSpacing([0.351562, 0.351562, 0.625]) #ElementType
    img.SetOrigin([-60, -110, -175]) #offset
    sitk.WriteImage(img, file_path)

In [ ]:
import os
import cv2
import glob
import SimpleITK
import numpy as np
from natsort import natsorted
from ipywidgets import interact
from matplotlib import pylab as plt
from skimage.util import random_noise
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

hdd = '/mnt/Dataset/TN/datasets/'
data_lists = natsorted(os.listdir('../Preprocess/GroundTruth/'))

mhd_slices = []
for index in natsorted(glob.glob('../Preprocess/GroundTruth/*/*.mhd')):
    mhd_slices.append(SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(index)).shape[0])

    
class dataProcess(object):

    def __init__(self, out_rows, out_cols, data_path  = "../Preprocess/mhd/",
                                           label_path = "../Preprocess/GT/*/",
                                           test_path  = "../Preprocess/mhd/",
                                           testlabel_path = "../Preprocess/GT/*/"):

        self.out_rows = out_rows
        self.out_cols = out_cols
        self.data_path = data_path
        self.label_path = label_path
        self.test_path = test_path
        self.testlabel_path = testlabel_path

    def _transform(self,imagein):
        image = imagein
        resize_image = cv2.resize(image, (512, 512), interpolation=cv2.INTER_NEAREST)
        if self.__channels and len(image.shape) < 3:  # make sure images are of shape(h,w,3)
            image = np.array([image for i in range(3)])
        else:
            resize_image = image
        return np.array(resize_image)

    def horizontal_flip(img):
        return img[:,::-1]

    def vertical_flip(img):
        return img[::-1,:]
        
    def upscale1_1_img(img):
        scale = 563     # floor(512*1.1)
        img = cv2.resize(img.astype(np.float64), (scale,scale), interpolation = cv2.INTER_NEAREST)
        img = img[25:537,25:537]
        img = img.astype(np.int32)
        return img

    def addnoise(img):
        mean = 0
        sigma = 50
        gauss = np.random.normal(mean,sigma,(512,512))
        img = img.astype(np.float64) + gauss
        img = img.astype(np.int32)
        return img
    
    def create_train_data(self, excepts=0):
        '''
        data_pathに存在する各症例を読み込み、rate倍のデータ拡張を行ったものを訓練データセットとする
        '''
        tag = data_lists[excepts]+"-"+data_lists[excepts+1]+"-"+data_lists[excepts+2]
        
        if not os.path.exists(hdd+"trainImages-except-"+tag+".npy"):
            print('Creating training images without #{} ...'.format(excepts, excepts+1, excepts+2))

            imgdatas= read_mhd_files(self.data_path, excepts=excepts )
            num = len(imgdatas)
            rate = 4
            final_images = np.ndarray([num*rate,1,512,512],'int32')
            imglabels = read_mhd_files(self.label_path, excepts=excepts)
            final_label = np.ndarray([num*rate,1,512,512],'int32')
            for i in tqdm(range(num)):
                final_images[rate*i,0] = imgdatas[i]
                final_images[rate*i+1,0] = dataProcess.horizontal_flip(imgdatas[i])
                final_images[rate*i+2,0] = dataProcess.vertical_flip(imgdatas[i])
                final_images[rate*i+3,0] = dataProcess.upscale1_1_img(imgdatas[i])
                final_label[rate*i,0] = imglabels[i]
                final_label[rate*i+1,0] = dataProcess.horizontal_flip(imglabels[i])
                final_label[rate*i+2,0] = dataProcess.vertical_flip(imglabels[i])
                final_label[rate*i+3,0] = dataProcess.upscale1_1_img(imglabels[i])


            print(final_images.shape)
            print(final_label.shape)
            np.save(hdd+"trainImages-except-"+data_lists[excepts]+"-"+data_lists[excepts+1]+"-"+data_lists[excepts+2]+".npy",final_images)
            np.save(hdd+"trainMasks-except-"+data_lists[excepts]+"-"+data_lists[excepts+1]+"-"+data_lists[excepts+2]+".npy", final_label)

            print('Saving to train.npy files done.')

    def create_test_data(self, choose=0):
        
        tag = data_lists[choose]+"-"+data_lists[choose+1]+"-"+data_lists[choose+2]
        
        if not os.path.exists(hdd+"testImages-"+ tag +".npy"):
            print("Creating testing images with #{} ...".format(choose, choose+1, choose+2))

            imgdatas =  read_mhd_files(self.test_path, choose=choose)
            num = len(imgdatas)
            final_images = np.ndarray([num, 1, 512, 512],'int32')

            imglabels =  read_mhd_files(self.testlabel_path, choose=choose)
            final_label = np.ndarray([num, 1, 512, 512],'int32')
            for i in tqdm(range(num)):
                final_images[i, 0] = imgdatas[i]
                final_label[i, 0] = imglabels[i]
            print(final_images.shape)
            print(final_label.shape)
            np.save(hdd+"testImages-"+ data_lists[choose]+"-"+data_lists[choose+1]+"-"+data_lists[choose+2]+".npy", final_images)
            np.save(hdd+"testMasks-"+ data_lists[choose]+"-"+data_lists[choose+1]+"-"+data_lists[choose+2]+".npy", final_label)

            print('Saving to test.npy files done.')



            
# datasets = dataProcess(512,512)
# datasets.create_train_data(excepts=0)
# datasets.create_test_data(choose=0)
# datasets = None  # メモリ開放

### 2. Learning and prediction

In [ ]:
from keras import backend as K

# GPU1つのみの設定
if 'tensorflow' == K.backend():
    import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "1"
set_session(tf.Session(config=config))

In [ ]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, LeakyReLU, UpSampling2D, MaxPooling2D, concatenate, BatchNormalization, Dropout
from keras import optimizers
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import SimpleITK as sitk

K.set_image_dim_ordering('th')  # Theano dimension ordering in this code

img_rows = 512
img_cols = 512
smooth = 1.



def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((2.0 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))


def dice_coef_np(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0-dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((1, img_rows, img_cols))
    #conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    #conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    #pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv1 = Conv2D(32, (3,3), padding = 'same')(inputs)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    #conv1 = BatchNormalization(momentum = 0.8)(conv1)
    conv1 = Conv2D(32, (3,3), padding = 'same')(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = BatchNormalization(momentum = 0.8)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    #conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    #conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    #pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv2 = Conv2D(64, (3,3), padding = 'same')(pool1)
    conv2 = LeakyReLU(alpha=0.2)(conv2)
    conv2 = BatchNormalization(momentum = 0.8)(conv2)
    conv2 = Conv2D(64, (3,3), padding = 'same')(conv2)
    conv2 = LeakyReLU(alpha=0.2)(conv2)
    conv2 = BatchNormalization(momentum = 0.8)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    #conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    #conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    #pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv3 = Conv2D(128, (3,3), padding = 'same')(pool2)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv3 = BatchNormalization(momentum = 0.8)(conv3)
    conv3 = Conv2D(128, (3,3), padding = 'same')(conv3)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv3 = BatchNormalization(momentum = 0.8)(conv3)    
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    #conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    #conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    #pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv4 = Conv2D(256, (3,3), padding = 'same')(pool3)
    conv4 = LeakyReLU(alpha=0.2)(conv4)
    conv4 = BatchNormalization(momentum = 0.8)(conv4)
    conv4 = Conv2D(256, (3,3), padding = 'same')(conv4)
    conv4 = LeakyReLU(alpha=0.2)(conv4)
    conv4 = BatchNormalization(momentum = 0.8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    #conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    #conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)
    conv5 = Conv2D(512, (3,3), padding = 'same')(pool4)
    conv5 = LeakyReLU(alpha=0.2)(conv5)
    conv5 = BatchNormalization(momentum = 0.8)(conv5)
    conv5 = Conv2D(512, (3,3), padding = 'same')(conv5)
    conv5 = LeakyReLU(alpha=0.2)(conv5)
    conv5 = BatchNormalization(momentum = 0.8)(conv5)
    ##up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1) %merge not exist in this version
    #up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis = 1)
    #conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    #conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis = 1)
    conv6 = Conv2D(256, (3,3), padding = 'same', activation='relu')(up6)
    #conv6 = Dropout(rate = 0.5)(conv6)   -> decrease the accuracy so removed 
    conv6 = BatchNormalization(momentum = 0.8)(conv6)
    conv6 = Conv2D(256, (3,3), padding = 'same', activation='relu')(conv6)
    #conv6 = Dropout(rate = 0.5)(conv6)   -> decrease the accuracy so removed 
    conv6 = BatchNormalization(momentum = 0.8)(conv6)

    ## up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    #up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis = 1)
    #conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    #conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis = 1)
    conv7 = Conv2D(128, (3,3), padding = 'same', activation='relu')(up7)
    conv7 = BatchNormalization(momentum = 0.8)(conv7)
    conv7 = Conv2D(128, (3,3), padding = 'same', activation='relu')(conv7)
    conv7 = BatchNormalization(momentum = 0.8)(conv7)
    ##up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    #up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=1)
    #conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    #conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis = 1)
    conv8 = Conv2D(64, (3,3), padding = 'same', activation='relu')(up8)
    conv8 = BatchNormalization(momentum = 0.8)(conv8)
    conv8 = Conv2D(64, (3,3), padding = 'same', activation='relu')(conv8)
    conv8 = BatchNormalization(momentum = 0.8)(conv8)

    ##up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    #up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=1)
    #conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    #conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis = 1)
    conv9 = Conv2D(32, (3,3), padding = 'same', activation='relu')(up9)
    conv9 = BatchNormalization(momentum = 0.8)(conv9)
    conv9 = Conv2D(32, (3,3), padding = 'same', activation='relu')(conv9)
    conv9 = BatchNormalization(momentum = 0.8)(conv9)
    #conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    #model.compile(optimizer=Adam(lr=1.0e-5), loss=dice_coef_loss, metrics=[dice_coef])  #original
    #model.compile(optimizer=Adam(lr=0.001, ), loss=dice_coef_loss, metrics=[dice_coef])
    sgd = optimizers.SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss=dice_coef_loss, metrics=[dice_coef])
    
    model.summary()
    
    return model


def train(use_existing, combination=0):
        
    # --------------- Training ----------------- #
    print('Loading train data...')
    tag = data_lists[combination]+"-"+data_lists[combination+1]+"-"+data_lists[combination+2]

    imgs_train = np.load(hdd+"trainImages-except-"+ tag +".npy").astype(np.float32)
    imgs_mask_train = np.load(hdd+"trainMasks-except-"+ tag +".npy").astype(np.float32)

    print('preprocessing train data...')

    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean  # images should already be standardized, but just in case
    imgs_train /= std

    print('constructing model...')

    model = get_unet()
    model_checkpoint = ModelCheckpoint('unet-'+tag+'.hdf5', monitor='loss', save_best_only=True)

    if use_existing:
        model.load_weights('./unet-'+tag+'.hdf5')

    print('Fitting model...')

    # add validation split
    # model.fit(imgs_train, imgs_mask_train, batch_size=4, epochs=30, verbose=1, validation_split = 0.1, shuffle=True, callbacks=[model_checkpoint])
    model.fit(imgs_train, imgs_mask_train, batch_size=4, epochs=20, verbose=1, validation_split = 0.1, shuffle=True, callbacks=[model_checkpoint])

    
    # plot figure
    #     plt.plot(range(1, NUM_EPOCH+1), learning_history.history['dice_coef'], label="training")
    #     # plt.plot(range(1, NUM_EPOCH+1), learning_history.history['val_dice_coef'], label="validation")
    #     plt.xlabel('Epochs')
    #     plt.ylabel('Accuracy')
    #     plt.legend()
    #     plt.ylim(0,1)
    #     plt.grid(axis='y', color='lightgray')
    #     plt.savefig('progress-'+str(combination)+'.png')
    #     plt.clf()
    
def predict(use_existing, combination=0):

    # --------------- Predict ----------------- #
    print('Loading test data...')
    
    #tag = glob.glob(mydata.testlabel_path)[0].split('/')[-2]
    tag = data_lists[combination]+"-"+data_lists[combination+1]+"-"+data_lists[combination+2]
    imgs_test = np.load(hdd+"testImages-"+ tag +".npy").astype(np.float32)
    imgs_mask = np.load(hdd+"testMasks-"+ tag +".npy").astype(np.float32)
    
    print('preprocessing test data...')

    mean = np.mean(imgs_test)  # mean for data centering
    std = np.std(imgs_test)  # std for data normalization
    
    imgs_test -= mean  # images should already be standardized, but just in case
    imgs_test /= std      
    
    model = get_unet()
    model.load_weights('./unet-'+tag+'.hdf5')
   
    num_test = len(imgs_test)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512], dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([imgs_test[i:i + 1]], verbose=0)[0]
            
    np.save(tag+'_masksTestPredict.npy', imgs_mask_test)
    mean = 0.0
    for i in range(num_test):
        mean += dice_coef_np(imgs_mask[i, 0], imgs_mask_test[i, 0])
    mean /= num_test
    print("Mean Dice Coeff : ", mean)
    
    total_img = np.reshape(imgs_mask_test,(num_test,512,512))
    each_img1 = total_img[0:mhd_slices[combination]]
    each_img2 = total_img[ mhd_slices[combination] : (mhd_slices[combination]+mhd_slices[combination+1])]
    each_img3 = total_img[(mhd_slices[combination]+mhd_slices[combination+1]) : ]
    
    save_mhd_files(each_img1, './result/result_mask_'+data_lists[combination]  +'_{:.2f}dice.mhd'.format(mean))
    save_mhd_files(each_img2, './result/result_mask_'+data_lists[combination+1]+'_{:.2f}dice.mhd'.format(mean))
    save_mhd_files(each_img3, './result/result_mask_'+data_lists[combination+2]+'_{:.2f}dice.mhd'.format(mean))
    
    # imgs_mask_test = np.squeeze(imgs_mask_test, axis=3)
    # np.save('imgs_mask_test.npy', imgs_mask_test)


    
# train_and_predict(False)





# 3つおきに 0->3->6->9->12
for index in range(0, len(os.listdir('../Preprocess/GroundTruth/')), 3):
    # データセット逐次生成
    datasets = dataProcess(512,512)
    datasets.create_train_data(excepts=index)
    datasets.create_test_data(choose=index)
    datasets = None  # メモリ開放

    train(False, combination=index)
    predict(False, combination=index)



### 3. Check Results

In [ ]:
import glob
import SimpleITK
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline


test  = SimpleITK.ReadImage(natsorted(glob.glob('./result/*.mhd'))[0])
label = SimpleITK.ReadImage((natsorted(glob.glob('../Preprocess/GroundTruth/*/*[!2].mhd'))[:12])[0])
test_array  = SimpleITK.GetArrayFromImage(test)
label_array = SimpleITK.GetArrayFromImage(label)

plt.figure(figsize=(20, 20), dpi=25)
plt.rcParams['figure.figsize'] = (10.0, 5.0)

@interact(slices=(0,test_array.shape[0]-1))
def plot_rolling_mean(slices=140):    
    plt.subplot(121)
    plt.imshow(label_array[slices, :, :])
    plt.title('GT')
    plt.subplot(122)
    plt.imshow(test_array[slices, :, :])
    plt.title('predict')
    
    plt.gray()
    plt.show()


In [ ]:
natsorted(glob.glob('../Preprocess/GroundTruth/*/*[!2].mhd'))[:12]

## test

In [ ]:
natsorted(os.listdir('../Preprocess/GroundTruth/'))

In [ ]:
def tes(excepts=None, choose=None):
    if (excepts) :print(excepts)

tes(1)

In [ ]:
a=0.005
print('{:.2f}'.format(a))

In [ ]:
lists= natsorted(glob.glob("./*Predict.npy"))
np.load(lists[0])

In [ ]:
a = []
for index in natsorted(glob.glob('../Preprocess/GroundTruth/*/*.mhd')):
    a.append(SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(index)).shape[0])

In [ ]:
natsorted(glob.glob(hdd+"*-*-*-*.npy"))

In [ ]:
del a[1:3]

In [ ]:
test_array.shape

In [ ]:
for index in range(0,len(os.listdir('../Preprocess/GroundTruth/')),3):
    print (index)